In [ ]:
!pip install wikipedia-api
!pip install datasets
import wikipediaapi
import re
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses


# 1. Wikipedia 크롤링
# 위키피디아 API 설정
wiki_wiki = wikipediaapi.Wikipedia('MyProject/1.0 (13wogns@gmail.com)', 'ko')

topics = ["공원", "도서관", "해변", "산", "도시", "마을", "강", "호수", "광장", "카페",
          "식당", "학교", "병원", "극장", "박물관", "시장", "공항", "체육관", "지하철역", "호텔",
          "초콜릿", "피자", "라면", "스시", "햄버거", "김치", "비빔밥", "떡볶이", "파스타", "스테이크",
          "샐러드", "치킨", "감자튀김", "샌드위치", "토스트", "오믈렛", "초밥", "케이크", "아이스크림", "컵라면",
          "의사", "변호사", "요리사", "교사", "프로그래머", "디자이너", "엔지니어", "간호사", "회계사", "군인",
          "경찰", "소방관", "조종사", "비서", "관리자", "연구원", "작가", "예술가", "음악가", "배우",
          "컴퓨터", "휴대폰", "책상", "의자", "시계", "텔레비전", "냉장고", "전자레인지", "세탁기", "전구",
          "수도꼭지", "마우스", "책상", "프린터", "카메라", "스피커", "이어폰", "헤드폰", "책", "노트",
          "해리포터", "슈퍼맨", "아이언맨", "스파이더맨", "신데렐라", "닥터 스트레인지", "가모라", "타노스", "배트맨", "원더우먼",
          "로켓 라쿤", "캡틴 아메리카", "토르", "헐크", "타노스", "데드풀", "엑스맨", "스파이더우먼", "레드후드", "가모라"]


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=af505e3be7146206987662ef9694cc29814f8fe4b2e0ac39e7d8f803fa1fabe8
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
word_text = {}
for word in topics:
    page = wiki_wiki.page(word)
    word_text[word] = page.text if page.exists() else "No summary available."

In [ ]:
# 2. 데이터 변환
word_data = pd.DataFrame(list(word_text.items()), columns=["word", "text"])
def keep_korean_and_period(text):
    # 한글, 띄어쓰기, 온점(.)만 남기고 나머지 문자 제거
    return re.sub(r'[^가-힣\s.]', '', text)

word_data['text'] = word_data['text'].apply(keep_korean_and_period)

In [ ]:
word_data

,word,text
0,공원,공원은 대중에게 개방되어 여러 사람들이 쉬거나 가벼운 운동 혹은 놀이를 즐길 수 있...
1,도서관,도서관 영어 은 책 잡지 영상 매체 마이크로필름 등의 다양한 자료를 공공 기관 단체...
2,해변,해변 또는 해빈은 해안선을 따라 파도와 연안류가 모래나 자갈 등을 쌓아올려서 만들어...
3,산,산은 주위보다 높이 솟아 있는 지형을 말한다. 한국어 고유어로는 뫼 또는 메라고 부...
4,도시,도시는 인간의 정치경제사회적인 활동이 중심이 되는 장소이며 인구 집중으로 인해 비교...
...,...,...
92,헐크,헐크는 마블 코믹스의 만화 캐릭터이다. 작가 스탠 리와 만화가 잭 커비가 창조한 캐...
93,데드풀,데드풀은 마블 코믹스 세계관의 등장인물로와 반영웅이다 웨이드 윈스턴 윌슨 이라는 ...
94,엑스맨,엑스맨영어 은 마블 코믹스에서 발행되는 코믹에 나오는 슈퍼히어로 팀이다. 또한 그들...
95,스파이더우먼,스파이더우먼의 다른 뜻은 다음과 같다.\n\n스파이더우먼 제시카 드루 오리지널 스파...


In [ ]:
# 카테고리 리스트
categories = ["장소", "음식", "직업", "물건", "인물"]

# 인덱스를 20으로 나눈 몫을 이용해 카테고리 할당
word_data['category'] = word_data.index.map(lambda i: categories[(i // 20) % len(categories)])

In [ ]:
# 공백 제거 및 빈 문장 제거
expanded_data = word_data.assign(text=word_data['text'].str.split('.')).explode('text')
expanded_data['text'] = expanded_data['text'].str.strip()
expanded_data = expanded_data[expanded_data['text'] != ""]

# word 값 변환
expanded_data['word'] = expanded_data['word']

# 인덱스 리셋
expanded_data = expanded_data.reset_index(drop=True)

# 결과 출력
word_data=expanded_data

word_data

,word,text,category
0,공원,공원은 대중에게 개방되어 여러 사람들이 쉬거나 가벼운 운동 혹은 놀이를 즐길 수 있...,장소
1,공원,공공녹지의 하나로서 국가나 지방자치단체 공공단체가 국민이나 주민의 보건 휴양 및 정...,장소
2,공원,공원은 인간의 즐거움과 휴양을 위해 또는 야생동물이나 자연 서식지를 보호하기 위해 ...,장소
3,공원,도시 공원은 마을과 도시 내부의 휴양을 위해 따로 마련한 녹지 공간이다,장소
4,공원,국립공원과 시골공원은 시골에서 휴양을 위해 사용되는 녹지 공간이다,장소
...,...,...,...
5061,레드후드,배트맨과 싸우는 파트너인 대 로빈이었지만 조커에게 살해당한 제이슨 토드는 다시 살아...,인물
5062,레드후드,년 유니버스의 리부트에 뒤이은 스토리라인 뉴 에는 레드 후드 갱이라는 범죄 조직이...,인물
5063,레드후드,제이슨 토드는 복귀한 이후로 현재까지 코믹스 연속성에서 레드 후드로 등장한다,인물
5064,레드후드,텔레비전 쇼 사이드 코믹스 등에서 사용되는 대체 연속성은 모든 조커 제이슨 토드 갱...,인물


In [ ]:
genre = "직업"
word_data_cat = word_data[word_data["category"] == genre]

In [ ]:
word_data_cat

,word,text,category
2189,의사를 한 문장으로 설명해줘.,의사 또는 는 현대의학의 전문가로서 인체의 질병 손상 각종 신체 혹은 정신의 ...,직업
2190,의사를 한 문장으로 설명해줘.,대한민국의 의료법에 의하면 의료진은 의사치과의사한의사조산사간호사로 나누고 있으며 이...,직업
2191,의사를 한 문장으로 설명해줘.,의사가 되고자 하는 자는 의과대학이나 의학전문대학원에서 의학을 전공하고 졸업하여 의...,직업
2192,의사를 한 문장으로 설명해줘.,다만 외국에서 의사 면허를 받은 대한민국의 국적을 갖고 영주권을 얻은 자는 소정의 ...,직업
2193,의사를 한 문장으로 설명해줘.,분류\n의과대학 혹은 의학전문대학원을 졸업하여 학위를 취득하고 의사면허시험을 통과하...,직업
...,...,...,...
2914,배우를 한 문장으로 설명해줘.,연기술과 배우훈련 연기자 만들기의 실체,직업
2915,배우를 한 문장으로 설명해줘.,집문당,직업
2916,배우를 한 문장으로 설명해줘.,년,직업
2917,배우를 한 문장으로 설명해줘.,같이 보기\n단역\n보조 출연자\n어린이 배우\n카메오 출연\n콤메디아 델라르테\n...,직업


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from datasets import Dataset

# ✅ 1. GPU 설정 (가능하면 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ 2. 모델 및 토크나이저 로드
model_name = "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # 🚀 GPU로 이동

# ✅ 3. 데이터셋 변환
dataset = Dataset.from_pandas(word_data)

# ✅ 4. 토크나이징 함수 정의
def tokenize_function(example):
    prompt = example["word"]
    response = example["text"]
    input_text = f"<s>{prompt}\n{response}</s>"

    tokenized = tokenizer(input_text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")

    return {
        "input_ids": tokenized["input_ids"].squeeze(0),
        "attention_mask": tokenized["attention_mask"].squeeze(0)
    }

# ✅ 5. 데이터셋을 PyTorch Tensor로 변환
tokenized_datasets = dataset.map(tokenize_function, batched=False)

# ✅ 6. DataLoader 설정
batch_size = 8  # 🚀 GPU를 사용하므로 배치 크기 증가 가능
train_dataloader = DataLoader(tokenized_datasets, batch_size=batch_size, shuffle=True)

# ✅ 7. 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# ✅ 8. 직접 학습 루프 구현
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        optimizer.zero_grad()

        # ✅ 배치를 Tensor로 변환 후 GPU 이동
        input_ids = torch.stack(batch["input_ids"]).to(device)
        attention_mask = torch.stack(batch["attention_mask"]).to(device)

        # ✅ 모델에 입력
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        # ✅ 손실 계산 및 역전파
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # ✅ 로그 출력
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# ✅ 9. 모델 저장
model.save_pretrained("./koreanlm-finetune")
tokenizer.save_pretrained("./koreanlm-finetune")

print("🎉 학습 완료! 모델이 ./koreanlm-finetune 폴더에 저장되었습니다.")


✅ Using device: cpu


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/5066 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
